In [1]:
# !pip install --quiet  datasets #to access squad dataset
# !pip install --quiet pyarrow   #to deal with parquet files for saving dataset if required
# !pip install --quiet  tqdm     #for progress bars
# !pip install --quiet transformers # for t5 model
# !pip install --quiet tokenizers  #tokenizers from HuggingFace
# !pip install --quiet sentencepiece #subword tokenizer used by T5
# !pip install --quiet pytorch-lightning # pytorch wrapper 
# !pip install --quiet torchtext # text utilities

# Contructed random 50% dataset

In [2]:
#imports
import pandas as pd
import torch
from tqdm import tqdm
from datasets import DatasetDict, Dataset, load_from_disk
# from torch.utils.data import Dataset, DataLoader
from pprint import pprint
import copy
import numpy as np
from collections import defaultdict
import ipdb, re

pd.options.display.max_rows , pd.options.display.max_columns  = 100,100  

In [20]:
# DOCTEAT
df_train_tdm_docteat_f1_all_templates = pd.read_parquet('../data/df_train_tdm_docteat_f1_all_templates.parquet')
df_dev_tdm_docteat_f1_all_templates = pd.read_parquet('../data/df_dev_tdm_docteat_f1_all_templates.parquet')

df_train_tdm_docteat_f2_all_templates = pd.read_parquet('../data/df_train_tdm_docteat_f2_all_templates.parquet')
df_dev_tdm_docteat_f2_all_templates = pd.read_parquet('../data/df_dev_tdm_docteat_f2_all_templates.parquet')

df_train_tdms_docteat_f1_all_templates = pd.read_parquet('../data/df_train_tdms_docteat_f1_all_templates.parquet')
df_dev_tdms_docteat_f1_all_templates = pd.read_parquet('../data/df_dev_tdms_docteat_f1_all_templates.parquet')

df_train_tdms_docteat_f2_all_templates = pd.read_parquet('../data/df_train_tdm_docteat_f2_all_templates.parquet')
df_dev_tdms_docteat_f2_all_templates = pd.read_parquet('../data/df_dev_tdm_docteat_f2_all_templates.parquet')

## LONG
# df_train_tdm_long_f1_all_templates = pd.read_parquet('../data/df_train_tdm_long_f1_all_templates.parquet')
# df_dev_tdm_long_f1_all_templates = pd.read_parquet('../data/df_dev_tdm_long_f1_all_templates.parquet')

# df_train_tdm_long_f2_all_templates = pd.read_parquet('../data/df_train_tdm_long_f2_all_templates.parquet')
# df_dev_tdm_long_f2_all_templates = pd.read_parquet('../data/df_dev_tdm_long_f2_all_templates.parquet')

# df_train_tdms_long_f1_all_templates = pd.read_parquet('../data/df_train_tdms_long_f1_all_templates.parquet')
# df_dev_tdms_long_f1_all_templates = pd.read_parquet('../data/df_dev_tdms_long_f1_all_templates.parquet')

# df_train_tdms_long_f2_all_templates = pd.read_parquet('../data/df_train_tdms_long_f2_all_templates.parquet')
# df_dev_tdms_long_f2_all_templates = pd.read_parquet('../data/df_dev_tdms_long_f2_all_templates.parquet')

# df_train_tdm_docteat_f1_all_templates.head()

In [4]:
df_train_tdm_docteat_f1_all_templates.describe()

,prompt,answer
count,82680,82680
unique,82650,2821
top,Read this and answer the question. If the ques...,unanswerable
freq,2,28080


In [4]:
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
# substring_to_match_squad_1

In [5]:
# Filter rows where the 'text' column contains the given substring
# substring_to_match_squad_1 = re.escape('Please answer a question about this article. If the question is unanswerable, say "unanswerable".')
substring_to_match_squad_1 = 'Please answer a question about this article. If the question is unanswerable, say "unanswerable".'
substring_to_match_squad_2 = 'Read this and answer the question. If the question is unanswerable, say \"unanswerable\".'
substring_to_match_squad_3 = '(If the question is unanswerable, say \"unanswerable\"'
substring_to_match_squad_4 = 'Try to answer this question if possible (otherwise reply \"unanswerable\")'
substring_to_match_squad_5 = 'If it is possible to answer this question, answer it for me (else, reply \"unanswerable\"):'
substring_to_match_squad_6 = 'Answer this question, if possible (if impossible, reply \"unanswerable\"):'
substring_to_match_squad_7 = 'What is the answer? (If it cannot be answered, return \"unanswerable\")'
substring_to_match_squad_8 = 'Now answer this question, if there is an answer (If it cannot be answered, return \"unanswerable\"):'

substring_to_match_drop_1 = 'Answer based on context:'
substring_to_match_drop_2 = 'Answer this question based on the article:'
# substring_to_match_drop_3 = ''
substring_to_match_drop_4 = 'Answer this question: '
substring_to_match_drop_5 = 'Read this article and answer this question'
substring_to_match_drop_6 = 'Based on the above article, answer a question.'
substring_to_match_drop_7 = 'Context: '


# df_train_tdm_docteat_f1_all_templates_squad_1 = df_train_tdm_docteat_f1_all_templates[
#     df_train_tdm_docteat_f1_all_templates['prompt'].str.contains(substring_to_match_squad_1, case=False, na=False)]

# df_train_tdm_docteat_f1_all_templates['template'] = df_train_tdm_docteat_f1_all_templates['prompt'].apply(
#     lambda x : "squad_1" if substring_to_match_squad_1 in x else "None")

df_train_tdm_docteat_f1_all_templates['template'] = df_train_tdm_docteat_f1_all_templates['prompt'].apply(
    lambda x : "squad_1" if substring_to_match_squad_1 in x else \
        "squad_2" if substring_to_match_squad_2 in x else \
        "squad_3" if substring_to_match_squad_3 in x else \
        "squad_4" if substring_to_match_squad_4 in x else \
        "squad_5" if substring_to_match_squad_5 in x else \
        "squad_6" if substring_to_match_squad_6 in x else \
        "squad_7" if substring_to_match_squad_7 in x else \
        "squad_8" if substring_to_match_squad_8 in x else \
        "drop_1" if substring_to_match_drop_1 in x else \
        "drop_2" if substring_to_match_drop_2 in x else \
        "drop_4" if substring_to_match_drop_4 in x else \
        "drop_5" if substring_to_match_drop_5 in x else \
        "drop_6" if substring_to_match_drop_6 in x else \
        "drop_7" if substring_to_match_drop_7 in x else "drop_3"
        )

# df_train_tdm_docteat_f1_all_templates['template'] = df_train_tdm_docteat_f1_all_templates['prompt'].apply(
#     lambda x : "squad_4" if re.search(f"{substring_to_match_squad_4}", x) else "None")

# df_train_tdm_docteat_f1_all_templates['template'] = df_train_tdm_docteat_f1_all_templates['prompt'].apply(
#     lambda x : "squad_1" if re.search(f"\b{substring_to_match_squad_1}\b", x) else \
#                 "squad_2" if re.search(f"\b{substring_to_match_squad_2}\b", x) else \
#                 "squad_3" if re.search(f"\b{substring_to_match_squad_3}\b", x) else "drop_3")


df_train_tdm_docteat_f1_all_templates.describe()

,prompt,answer,template
count,82680,82680,82680
unique,82650,2821,15
top,Read this and answer the question. If the ques...,unanswerable,squad_1
freq,2,28080,5512


In [6]:
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_1'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_2'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_3'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_4'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_6'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_7'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_8'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_1'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_2'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_3'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_4'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_7'].describe())

,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,squad_1
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,Read this and answer the question. If the ques...,unanswerable,squad_2
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,squad_3
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,squad_4
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,squad_5
freq,2,1872,5512


...

,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,drop_3
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable,drop_4
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,Read this article and answer this question Fix...,unanswerable,drop_5
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,Read this article and answer this question Fix...,unanswerable,drop_5
freq,2,1872,5512


,prompt,answer,template
count,5512,5512,5512
unique,5510,2821,1
top,Context: FixMatch: Simplifying Semi-Supervised...,unanswerable,drop_7
freq,2,1872,5512


In [ ]:
result_df  = pd.DataFrame(columns = ['prompt', 'answer'])   

records = df_train_tdm_docteat_f1_all_templates.to_dict("records")

In [12]:
df_train_tdm_docteat_f1_all_templates_squad_1 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_1'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_2 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_2'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_3 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_3'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_4 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_4'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_5 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_5'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_6 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_6'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_7 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_7'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_squad_8 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_8'].sample(frac=0.5)

df_train_tdm_docteat_f1_all_templates_drop_1 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_1'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_2 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_2'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_3 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_3'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_4 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_4'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_5 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_6 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_6'].sample(frac=0.5)
df_train_tdm_docteat_f1_all_templates_drop_7 = df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_7'].sample(frac=0.5)


df_train_tdm_docteat_f1_all_templates_50_percent = pd.concat([
    df_train_tdm_docteat_f1_all_templates_squad_1, 
    df_train_tdm_docteat_f1_all_templates_squad_2, 
    df_train_tdm_docteat_f1_all_templates_squad_3, 
    df_train_tdm_docteat_f1_all_templates_squad_4, 
    df_train_tdm_docteat_f1_all_templates_squad_5, 
    df_train_tdm_docteat_f1_all_templates_squad_6, 
    df_train_tdm_docteat_f1_all_templates_squad_7, 
    df_train_tdm_docteat_f1_all_templates_squad_8, 
    df_train_tdm_docteat_f1_all_templates_drop_1, 
    df_train_tdm_docteat_f1_all_templates_drop_2, 
    df_train_tdm_docteat_f1_all_templates_drop_3, 
    df_train_tdm_docteat_f1_all_templates_drop_4, 
    df_train_tdm_docteat_f1_all_templates_drop_5,
    df_train_tdm_docteat_f1_all_templates_drop_6,
    df_train_tdm_docteat_f1_all_templates_drop_7
    ], ignore_index=True)

df_train_tdm_docteat_f1_all_templates_50_percent.describe()

,prompt,answer,template
count,41340,41340,41340
unique,41331,2821,15
top,Read this and answer the question. If the ques...,unanswerable,squad_1
freq,2,14136,2756


In [11]:
15*2756

41340

In [ ]:
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_1'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_2'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_3'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_4'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_6'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_7'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='squad_8'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_1'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_2'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_3'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_4'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_5'].describe())
display(df_train_tdm_docteat_f1_all_templates[df_train_tdm_docteat_f1_all_templates['template']=='drop_7'].describe())

In [ ]:
# Randomly select 20% of the rows from the dataframe
df_sample = df.sample(frac=0.2)
print(df_sample)


In [9]:
df_train_tdm_docteat_f1_squad_1 = pd.read_parquet('../data/df_train_tdm_docteat_f1_squad_1.parquet')
df_train_tdm_docteat_f1_all_templates_squad_1.describe()

,prompt,answer
count,5512,5512
unique,5510,2821
top,FixMatch: Simplifying Semi-Supervised Learning...,unanswerable
freq,2,1872


In [ ]:


df_train_f1_drop_1.to_parquet('../data/df_train_tdm_docteat_f1_drop_1.parquet')
df_dev_f1_drop_1.to_parquet('../data/df_dev_tdm_docteat_f1_drop_1.parquet')
df_train_f2_drop_1.to_parquet('../data/df_train_tdm_docteat_f2_drop_1.parquet')
df_dev_f2_drop_1.to_parquet('../data/df_dev_tdm_docteat_f2_drop_1.parquet')